# Yolo-v2 DW2TF validation

In [1]:
import numpy as np
import tensorflow as tf

# Random input data to test 
in_data = np.random.randn(64, 608, 608, 3) * 120.56

### Output from Darkflow converted Yolo-v2

In [2]:
tf.reset_default_graph()

tf.train.import_meta_graph('/data/darkflow/yolov2.ckpt.meta')
ckpt_path = '/data/darkflow/yolov2.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('input:0')
out_t = g.get_tensor_by_name('output:0')

with tf.Session() as sess:
    var_list = {} # dictionary mapping variable name to tensor, used to create saver object to restore
    reader = tf.train.NewCheckpointReader(ckpt_path)
    for key in reader.get_variable_to_shape_map():
        # Look for all variables in ckpt that are used by the graph
        try:
            tensor = g.get_tensor_by_name(key + ":0")
        except KeyError:
            # This tensor doesn't exist in the graph (for example it's
            # 'global_step' or a similar housekeeping element) so skip it.
            continue
        var_list[key] = tensor
    saver = tf.train.Saver(var_list=var_list)
    saver.restore(sess, ckpt_path)

    out_data_darkflow = sess.run(out_t, feed_dict={in_t: in_data})

INFO:tensorflow:Restoring parameters from /data/darkflow/yolov2.ckpt


### Output from DW2TF converted Yolo-v2

In [3]:
tf.reset_default_graph()

tf.train.import_meta_graph('/data/DW2TF/data/yolov2.ckpt.meta')
ckpt_path = '/data/DW2TF/data/yolov2.ckpt'

g = tf.get_default_graph()
in_t = g.get_tensor_by_name('yolov2/net1:0')
out_t = g.get_tensor_by_name('yolov2/convolutional23/BiasAdd:0')

with tf.Session() as sess:
    var_list = {} # dictionary mapping variable name to tensor, used to create saver object to restore
    reader = tf.train.NewCheckpointReader(ckpt_path)
    for key in reader.get_variable_to_shape_map():
        # Look for all variables in ckpt that are used by the graph
        try:
            tensor = g.get_tensor_by_name(key + ":0")
        except KeyError:
            # This tensor doesn't exist in the graph (for example it's
            # 'global_step' or a similar housekeeping element) so skip it.
            continue
        var_list[key] = tensor
    saver = tf.train.Saver(var_list=var_list)
    saver.restore(sess, ckpt_path)
    
    out_data_dw2tf = sess.run(out_t, feed_dict={in_t: in_data})

INFO:tensorflow:Restoring parameters from /data/DW2TF/data/yolov2.ckpt


### RMS Error

In [4]:
print(out_data_darkflow.shape)
print(out_data_dw2tf.shape)
print(np.mean(out_data_darkflow - out_data_dw2tf)**2)

(64, 19, 19, 425)
(64, 19, 19, 425)
1.0358586146179594e-14


In [5]:
print(out_data_darkflow[0,0,0,:10])

[ -3.7852187    5.77546     -4.7733483   -3.2303553  -21.29596
  16.798992    17.598959    13.808735    -0.97558796 -14.386962  ]


In [6]:
print(out_data_dw2tf[0,0,0,:10])

[ -3.7852492    5.7753325   -4.7733445   -3.2302966  -21.295998
  16.798998    17.598883    13.8088255   -0.97567666 -14.386923  ]


# Reorg layer test

In [7]:
batch_size, height, width, channels = 1, 2, 4, 1
stride = 2

_height, _width, _channel = height // stride, width // stride, channels * stride * stride

input = np.array([[[[0], [2], [1], [3]], [[4], [6], [5], [7]]]])

net = input
net = tf.reshape(net, [batch_size, _height, stride, _width, stride, channels])
net = tf.transpose(net, [0, 1, 3, 2, 4, 5])  # batch_size, _height, _width, stride, stride, channels
net = tf.reshape(net, [batch_size, _height, _width, stride * stride * channels], name='reorg')

output = net.eval(session=tf.Session())

print(input.shape)
print(output.shape)

(1, 2, 4, 1)
(1, 1, 2, 4)


In [8]:
input

array([[[[0],
         [2],
         [1],
         [3]],

        [[4],
         [6],
         [5],
         [7]]]])

In [9]:
output

array([[[[0, 2, 4, 6],
         [1, 3, 5, 7]]]])

In [10]:
assert input[0,0,0,0] == output[0,0,0,0]
assert input[0,0,1,0] == output[0,0,0,1]
assert input[0,0,2,0] == output[0,0,1,0]
assert input[0,0,3,0] == output[0,0,1,1]
assert input[0,1,0,0] == output[0,0,0,2]
assert input[0,1,1,0] == output[0,0,0,3]
assert input[0,1,2,0] == output[0,0,1,2]
assert input[0,1,3,0] == output[0,0,1,3]

# Read darknet .weights

In [11]:
weight_array = np.fromfile('/data/darknet/yolov2.weights', dtype='f')

In [12]:
weight_array.shape

(50983565,)